In [1]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from scipy import ndimage
from keras.optimizers import Adadelta,Adam
from keras.callbacks import TensorBoard, CSVLogger, ModelCheckpoint
# from lipnet.lipreading.generators import BasicGenerator
# from lipnet.lipreading.callbacks import Statistics, Visualize
# from lipnet.lipreading.curriculums import Curriculum
# from lipnet.core.decoders import Decoder
# from lipnet.lipreading.helpers import labels_to_text
# from lipnet.utils.spell import Spell
from lipnet.model2 import LipNet
import numpy as np
import datetime
import cv2
import skvideo.io
import dlib
from scipy.misc import imresize
# from lipnet.lipreading.aligns import Align
import imageio
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
import random
import tensorflow as tf
import keras.backend as K
tf_config = tf.ConfigProto(allow_soft_placement=False)
tf_config.gpu_options.allow_growth = True
s = tf.Session(config=tf_config)
K.set_session(s)

FACE_PREDICTOR_PATH  = os.path.join('common','predictors','shape_predictor_68_face_landmarks.dat')
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(FACE_PREDICTOR_PATH)

frames_count = 30
words_folder = 'training/unseen_speakers/datasets/lipread_mp4'
words = os.listdir(words_folder)
words = ['Building','Absolutely','Winds'
,'Figures'
,'Years'
# ,'Series'
# ,'Given'
# ,'Questions'
# ,'Phone'
# ,'Justice'
# ,'Human'
# ,'Powers'
# ,'Former'
# ,'Should'
# ,'Guilty'
,'Having'
,'Their'
,'Often'
,'Facing'
,'Every'] #words[:2]
words = [s.upper() for s in words]
max_len = 12 #np.max([len(word) for word in words])
used_words = {''}
print(words)
def datagen(data_partition = 'train', batch_size = 16):
    while True:
        counter = 0
        word_idx = np.random.randint(len(words))
        used_words.add(words[word_idx])
        train_folder = 'training/unseen_speakers/datasets/lipread_mp4/{0}/{1}/'.format(words[word_idx],data_partition)
    #         print(train_folder)
        files = os.listdir(train_folder)
        paths = list(set([x.split('.')[0] for x in files]))
        
        X_data = []#np.zeros((batch_size,))
        Y_data = []
        input_length = []
        label_length = []
        source_str = []
        while counter<batch_size:
#             print(counter)
            idx = np.random.randint(len(paths))
            path = os.path.join(train_folder,paths[idx]+'.mp4')
    #         src_str_path = os.path.join(train_folder,paths[idx]+'.txt')
    #         with open(src_str_path) as f:
    #             l = f.readlines()
    #             print(l)

            vid = imageio.get_reader(path,  'ffmpeg')
            frames = []
            end = 0
            for i in range(frames_count):
                end = i
                frames.append(vid.get_data(i))

            ###
#             print('vedio frame shape:',np.shape(frames), data_partition, paths[idx])
            frames = get_frames_mouth(detector,predictor, frames, paths[idx])
            
            # for completely turned faces dlib cant identify the lip hence cannot crop
            # this check ensures only vid croppings which are 50x100x3 are added in batch
#             print('mouth frame shape:',np.shape(frames), data_partition, paths[idx])

            if 'numpy.ndarray' in str(type(frames)) \
                    and len(np.shape(frames))==4 \
                    and np.shape(frames)[1] == 100 \
                    and np.shape(frames)[2] == 50:
            
                X_data.append(frames)


                ####
                Y_data.append([ord(c) - 65 for c in words[word_idx]]+[-1]*(max_len-len(words[word_idx])))
                input_length.append(frames_count)
                label_length.append(len(words[word_idx]))
                source_str.append(words[word_idx])
                counter+=1
            else:
                continue
        
        X_data = np.array(X_data).astype(np.float32) / 255 
        Y_data = np.array(Y_data)
        input_length = np.array(input_length)
        label_length = np.array(label_length)
        source_str = np.array(source_str)
        
        inputs = {'the_input': X_data,
                  'the_labels': Y_data,
                  'input_length': input_length,
                  'label_length': label_length,
                  'source_str': source_str  # used for visualization only
                  }
        outputs = {'ctc': np.zeros([batch_size])}
        
#         print('the_input:', inputs['the_input'].shape)
#         print('the_labels:', inputs['the_labels'].shape)
#         print('input_length:', inputs['input_length'].shape)
#         print('label_length:', inputs['label_length'].shape)
#         print('source_str:', inputs['source_str'].shape)
#         print('batch_frames_shape',X_data.shape, data_partition)
        yield (inputs, outputs)

['BUILDING', 'ABSOLUTELY', 'WINDS', 'FIGURES', 'YEARS', 'HAVING', 'THEIR', 'OFTEN', 'FACING', 'EVERY']


In [3]:

def get_frames_mouth(detector, predictor, frames, word):
    try:
        MOUTH_WIDTH = 100
        MOUTH_HEIGHT = 50
        HORIZONTAL_PAD = 0.19
        normalize_ratio = None
        mouth_frames = []
        for frame in frames:
            dets = detector(frame, 1)
            shape = None
            for k, d in enumerate(dets):
                shape = predictor(frame, d)
                i = -1
            if shape is None: # Detector doesn't detect face, just return as is
                return frames
            mouth_points = []
            for part in shape.parts():
                i += 1
                if i < 48: # Only take mouth region
                    continue
                mouth_points.append((part.x,part.y))
            np_mouth_points = np.array(mouth_points)

            mouth_centroid = np.mean(np_mouth_points[:, -2:], axis=0)

            if normalize_ratio is None:
                mouth_left = np.min(np_mouth_points[:, :-1]) * (1.0 - HORIZONTAL_PAD)
                mouth_right = np.max(np_mouth_points[:, :-1]) * (1.0 + HORIZONTAL_PAD)

                normalize_ratio = MOUTH_WIDTH / float(mouth_right - mouth_left)

            new_img_shape = (int(frame.shape[0] * normalize_ratio), int(frame.shape[1] * normalize_ratio))
            resized_img = imresize(frame, new_img_shape)

            mouth_centroid_norm = mouth_centroid * normalize_ratio

            mouth_l = int(mouth_centroid_norm[0] - MOUTH_WIDTH / 2)
            mouth_r = int(mouth_centroid_norm[0] + MOUTH_WIDTH / 2)
            mouth_t = int(mouth_centroid_norm[1] - MOUTH_HEIGHT / 2)
            mouth_b = int(mouth_centroid_norm[1] + MOUTH_HEIGHT / 2)

            mouth_crop_image = resized_img[mouth_t:mouth_b, mouth_l:mouth_r]
            
            mouth_frames.append(ndimage.rotate(mouth_crop_image, 90))
    #     print('shape mouth frames:',np.shape(mouth_frames))
        return np.array(mouth_frames)
    except Exception as e:
        print(e)
        print(word)
        return []

In [4]:
# global lip_gen

run_name = datetime.datetime.now().strftime('%Y:%m:%d_%H:%M:%S')
run_name, start_epoch, stop_epoch, img_c, img_w, img_h, frames_n, absolute_max_string_len, minibatch_size = \
    run_name, 0, 1000, 3, 100, 50, 30, 12, 2



CURRENT_PATH = os.getcwd()+'/training/unseen_speakers'
DATASET_DIR  = os.path.join(CURRENT_PATH, 'datasets')
OUTPUT_DIR   = os.path.join(CURRENT_PATH, 'results')
LOG_DIR      = os.path.join(CURRENT_PATH, 'logs')
# if not os.path.exists('training/unseen_speakers/results/'+run_name):
#     os.makedirs('training/unseen_speakers/results/'+run_name)
if not os.path.exists('training/unseen_speakers/logs/'+run_name):
    os.makedirs('training/unseen_speakers/logs/'+run_name)

PREDICT_GREEDY      = False
PREDICT_BEAM_WIDTH  = 200
PREDICT_DICTIONARY  = os.path.join(CURRENT_PATH,'..','..','common','dictionaries','lfw.txt')
FACE_PREDICTOR_PATH  = os.path.join(CURRENT_PATH,'..','..','common','predictors','shape_predictor_68_face_landmarks.dat')

# def curriculum_rules(epoch):
#     return { 'sentence_length': -1, 'flip_probability': 0.5, 'jitter_probability': 0.05 }


# # def train(run_name, start_epoch, stop_epoch, img_c, img_w, img_h, frames_n, absolute_max_string_len, minibatch_size):
# curriculum = Curriculum(curriculum_rules)
# lip_gen = BasicGenerator(dataset_path=DATASET_DIR, #vtype = "face", face_predictor_path = FACE_PREDICTOR_PATH,
#                             minibatch_size=minibatch_size,
#                             img_c=img_c, img_w=img_w, img_h=img_h, frames_n=frames_n,
#                             absolute_max_string_len=absolute_max_string_len,
#                             curriculum=curriculum, start_epoch=start_epoch).build()
# lip_gen_train = datagen(data_partition='train', batch_size=5)
# lip_gen_val = datagen(data_partition='val', batch_size=5)

lipnet = LipNet(img_c=img_c, img_w=img_w, img_h=img_h, frames_n=frames_n,
                        absolute_max_string_len=absolute_max_string_len, output_size=27)
# lipnet.summary()
# adadelta = Adadelta(lr=0.0001, decay=1e-08, epsilon=1e-08)

adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=1e-08)

# the loss calc occurs elsewhere, so use a dummy lambda func for the loss
lipnet.model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=adam, metrics=['accuracy'])


# spell = Spell(path=PREDICT_DICTIONARY)
# decoder = Decoder(greedy=PREDICT_GREEDY, beam_width=PREDICT_BEAM_WIDTH,
#                   postprocessors=[labels_to_text, spell.sentence])

# define callbacks
# statistics  = Statistics(lipnet, lip_gen.next_val(), decoder, 256, output_dir=os.path.join(OUTPUT_DIR, run_name))
# visualize   = Visualize(os.path.join(OUTPUT_DIR, run_name), lipnet, lip_gen.next_val(), decoder, num_display_sentences=minibatch_size)
tensorboard = TensorBoard(log_dir=os.path.join(LOG_DIR, 'twenty-'+run_name))
# csv_logger  = CSVLogger(os.path.join(LOG_DIR, "{}-{}.csv".format('training',run_name)), separator=',', append=True)
checkpoint  = ModelCheckpoint(save_best_only=True,
                              filepath='training/unseen_speakers/results/weights_lfw-ten.hdf5'
                              , monitor='val_loss'
                              , verbose=1
                              , mode='min'
                              , period=1)
    
    # x = next(lip_gen.next_val())
    # for i in x:
    #     print('$$$$$----------', type(i))





y_pred (?, 30, 27)


In [5]:
# run_name = datetime.datetime.now().strftime('%Y:%m:%d:%H:%M:%S')
# train(run_name, 0, 1000, 3, 100, 50, 75, 32, 2)

In [6]:
# os.getcwd()+'training/unseen_speakers'

In [7]:
# g=lip_gen.next_train()
# inputs, outputs = next(g)

In [8]:
# inputs

In [9]:
# len('place green at t six now')

In [10]:
# len([ 15.,  11.,   0.,   2.,   4.,  26.,   6.,  17.,   4.,   4.,  13.,
#           26.,   0.,  19.,  26.,  19.,  26.,  18.,   8.,  23.,  26.,  13.,
#           14.,  22.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.])

In [11]:
# outputs

In [12]:
# load weight if necessary

start_epoch = 17084
if start_epoch > 0:
    weight_file = os.path.join(OUTPUT_DIR, 'weights_lfw-ten.hdf5')
    lipnet.model.load_weights(weight_file)

lipnet.model.fit_generator(generator=datagen(data_partition='train', batch_size=4),
                    steps_per_epoch=1
                    , epochs=50000,
                    #steps_per_epoch=lip_gen.default_training_steps, epochs=stop_epoch,
                    validation_data=datagen(data_partition='val', batch_size=4)
                    , validation_steps=1,#validation_steps=lip_gen.default_validation_steps,
                    callbacks=[checkpoint,tensorboard],                         
                    initial_epoch=start_epoch, 
                    verbose=1)
                    #max_q_size=5,
                    #workers=2,
                    #pickle_safe=True)

Epoch 17085/50000
1/1 [==============================] - 15s 15s/step - loss: 4.1436 - acc: 0.0000e+00 - val_loss: 17.8083 - val_acc: 0.0000e+00

Epoch 17085: val_loss improved from inf to 17.80834, saving model to training/unseen_speakers/results/weights_lfw-ten.hdf5
Epoch 17086/50000
1/1 [==============================] - 7s 7s/step - loss: 12.7777 - acc: 0.0000e+00 - val_loss: 12.8267 - val_acc: 0.0000e+00

Epoch 17086: val_loss improved from 17.80834 to 12.82666, saving model to training/unseen_speakers/results/weights_lfw-ten.hdf5
Epoch 17087/50000
1/1 [==============================] - 10s 10s/step - loss: 11.3588 - acc: 0.0000e+00 - val_loss: 12.2712 - val_acc: 0.0000e+00

Epoch 17087: val_loss improved from 12.82666 to 12.27119, saving model to training/unseen_speakers/results/weights_lfw-ten.hdf5
Epoch 17088/50000
1/1 [==============================] - 10s 10s/step - loss: 13.6864 - acc: 0.0000e+00 - val_loss: 9.7022 - val_acc: 0.0000e+00

Epoch 17088: val_loss improved from 1


Epoch 17123: val_loss did not improve from 5.89972
Epoch 17124/50000
1/1 [==============================] - 11s 11s/step - loss: 10.9745 - acc: 0.0000e+00 - val_loss: 16.1313 - val_acc: 0.0000e+00

Epoch 17124: val_loss did not improve from 5.89972
Epoch 17125/50000
1/1 [==============================] - 11s 11s/step - loss: 11.0904 - acc: 0.0000e+00 - val_loss: 6.7922 - val_acc: 0.0000e+00

Epoch 17125: val_loss did not improve from 5.89972
Epoch 17126/50000
1/1 [==============================] - 11s 11s/step - loss: 6.6897 - acc: 0.0000e+00 - val_loss: 9.4012 - val_acc: 0.0000e+00

Epoch 17126: val_loss did not improve from 5.89972
Epoch 17127/50000
1/1 [==============================] - 11s 11s/step - loss: 8.6141 - acc: 0.0000e+00 - val_loss: 6.2552 - val_acc: 0.0000e+00

Epoch 17127: val_loss did not improve from 5.89972
Epoch 17128/50000
1/1 [==============================] - 11s 11s/step - loss: 9.4304 - acc: 0.0000e+00 - val_loss: 21.7106 - val_acc: 0.0000e+00

Epoch 17128: va

1/1 [==============================] - 11s 11s/step - loss: 10.1642 - acc: 0.0000e+00 - val_loss: 9.7866 - val_acc: 0.0000e+00

Epoch 17165: val_loss did not improve from 3.82616
Epoch 17166/50000
1/1 [==============================] - 11s 11s/step - loss: 10.5696 - acc: 0.0000e+00 - val_loss: 16.0988 - val_acc: 0.0000e+00

Epoch 17166: val_loss did not improve from 3.82616
Epoch 17167/50000
1/1 [==============================] - 11s 11s/step - loss: 10.7870 - acc: 0.0000e+00 - val_loss: 9.7858 - val_acc: 0.0000e+00

Epoch 17167: val_loss did not improve from 3.82616
Epoch 17168/50000
1/1 [==============================] - 11s 11s/step - loss: 8.7133 - acc: 0.0000e+00 - val_loss: 12.1816 - val_acc: 0.0000e+00

Epoch 17168: val_loss did not improve from 3.82616
Epoch 17169/50000
1/1 [==============================] - 13s 13s/step - loss: 9.0393 - acc: 0.0000e+00 - val_loss: 11.8703 - val_acc: 0.0000e+00

Epoch 17169: val_loss did not improve from 3.82616
Epoch 17170/50000
1/1 [=========


Epoch 17206: val_loss did not improve from 3.82616
Epoch 17207/50000
1/1 [==============================] - 11s 11s/step - loss: 5.1063 - acc: 0.0000e+00 - val_loss: 8.7068 - val_acc: 0.0000e+00

Epoch 17207: val_loss did not improve from 3.82616
Epoch 17208/50000
1/1 [==============================] - 11s 11s/step - loss: 14.6142 - acc: 0.0000e+00 - val_loss: 11.9729 - val_acc: 0.0000e+00

Epoch 17208: val_loss did not improve from 3.82616
Epoch 17209/50000
1/1 [==============================] - 11s 11s/step - loss: 10.3565 - acc: 0.0000e+00 - val_loss: 9.5675 - val_acc: 0.0000e+00

Epoch 17209: val_loss did not improve from 3.82616
Epoch 17210/50000
1/1 [==============================] - 11s 11s/step - loss: 9.3840 - acc: 0.0000e+00 - val_loss: 12.1902 - val_acc: 0.0000e+00

Epoch 17210: val_loss did not improve from 3.82616
Epoch 17211/50000
1/1 [==============================] - 11s 11s/step - loss: 17.4335 - acc: 0.0000e+00 - val_loss: 9.8001 - val_acc: 0.0000e+00

Epoch 17211: v


Epoch 17248: val_loss did not improve from 3.82616
Epoch 17249/50000
1/1 [==============================] - 11s 11s/step - loss: 14.2100 - acc: 0.0000e+00 - val_loss: 12.0560 - val_acc: 0.0000e+00

Epoch 17249: val_loss did not improve from 3.82616
Epoch 17250/50000
1/1 [==============================] - 11s 11s/step - loss: 11.8496 - acc: 0.0000e+00 - val_loss: 12.0792 - val_acc: 0.0000e+00

Epoch 17250: val_loss did not improve from 3.82616
Epoch 17251/50000
1/1 [==============================] - 11s 11s/step - loss: 10.6090 - acc: 0.0000e+00 - val_loss: 20.2990 - val_acc: 0.0000e+00

Epoch 17251: val_loss did not improve from 3.82616
Epoch 17252/50000
1/1 [==============================] - 11s 11s/step - loss: 8.7828 - acc: 0.0000e+00 - val_loss: 9.1299 - val_acc: 0.0000e+00

Epoch 17252: val_loss did not improve from 3.82616
Epoch 17253/50000
1/1 [==============================] - 11s 11s/step - loss: 7.9410 - acc: 0.0000e+00 - val_loss: 8.7317 - val_acc: 0.0000e+00

Epoch 17253: 

...


Epoch 22080: val_loss did not improve from 1.91368
Epoch 22081/50000
1/1 [==============================] - 11s 11s/step - loss: 5.9622 - acc: 0.0000e+00 - val_loss: 6.4875 - val_acc: 0.0000e+00

Epoch 22081: val_loss did not improve from 1.91368
Epoch 22082/50000
1/1 [==============================] - 11s 11s/step - loss: 8.1428 - acc: 0.0000e+00 - val_loss: 11.4162 - val_acc: 0.0000e+00

Epoch 22082: val_loss did not improve from 1.91368
Epoch 22083/50000
1/1 [==============================] - 11s 11s/step - loss: 12.3980 - acc: 0.0000e+00 - val_loss: 9.3435 - val_acc: 0.0000e+00

Epoch 22083: val_loss did not improve from 1.91368
Epoch 22084/50000
1/1 [==============================] - 11s 11s/step - loss: 10.4061 - acc: 0.0000e+00 - val_loss: 9.2686 - val_acc: 0.0000e+00

Epoch 22084: val_loss did not improve from 1.91368
Epoch 22085/50000
1/1 [==============================] - 11s 11s/step - loss: 8.1782 - acc: 0.0000e+00 - val_loss: 10.4282 - val_acc: 0.0000e+00

Epoch 22085: va


Epoch 22122: val_loss did not improve from 1.91368
Epoch 22123/50000
1/1 [==============================] - 13s 13s/step - loss: 13.6518 - acc: 0.0000e+00 - val_loss: 15.1030 - val_acc: 0.0000e+00

Epoch 22123: val_loss did not improve from 1.91368
Epoch 22124/50000
1/1 [==============================] - 11s 11s/step - loss: 12.8975 - acc: 0.0000e+00 - val_loss: 5.7050 - val_acc: 0.0000e+00

Epoch 22124: val_loss did not improve from 1.91368
Epoch 22125/50000
1/1 [==============================] - 13s 13s/step - loss: 13.1767 - acc: 0.0000e+00 - val_loss: 11.9304 - val_acc: 0.0000e+00

Epoch 22125: val_loss did not improve from 1.91368
Epoch 22126/50000
1/1 [==============================] - 14s 14s/step - loss: 11.3810 - acc: 0.0000e+00 - val_loss: 10.2886 - val_acc: 0.0000e+00

Epoch 22126: val_loss did not improve from 1.91368
Epoch 22127/50000
1/1 [==============================] - 11s 11s/step - loss: 4.6982 - acc: 0.0000e+00 - val_loss: 4.0075 - val_acc: 0.0000e+00

Epoch 22127:


Epoch 22164: val_loss did not improve from 1.91368
Epoch 22165/50000
1/1 [==============================] - 10s 10s/step - loss: 9.3030 - acc: 0.0000e+00 - val_loss: 15.0738 - val_acc: 0.0000e+00

Epoch 22165: val_loss did not improve from 1.91368
Epoch 22166/50000
1/1 [==============================] - 11s 11s/step - loss: 13.6540 - acc: 0.0000e+00 - val_loss: 11.6786 - val_acc: 0.0000e+00

Epoch 22166: val_loss did not improve from 1.91368
Epoch 22167/50000
1/1 [==============================] - 11s 11s/step - loss: 10.4849 - acc: 0.0000e+00 - val_loss: 8.4205 - val_acc: 0.0000e+00

Epoch 22167: val_loss did not improve from 1.91368
Epoch 22168/50000
1/1 [==============================] - 11s 11s/step - loss: 8.5446 - acc: 0.0000e+00 - val_loss: 10.0950 - val_acc: 0.0000e+00

Epoch 22168: val_loss did not improve from 1.91368
Epoch 22169/50000
1/1 [==============================] - 11s 11s/step - loss: 15.2989 - acc: 0.0000e+00 - val_loss: 11.4661 - val_acc: 0.0000e+00

Epoch 22169:


Epoch 22206: val_loss did not improve from 1.91368
Epoch 22207/50000
1/1 [==============================] - 12s 12s/step - loss: 12.4868 - acc: 0.0000e+00 - val_loss: 9.5237 - val_acc: 0.0000e+00

Epoch 22207: val_loss did not improve from 1.91368
Epoch 22208/50000
1/1 [==============================] - 12s 12s/step - loss: 7.3120 - acc: 0.0000e+00 - val_loss: 7.3236 - val_acc: 0.0000e+00

Epoch 22208: val_loss did not improve from 1.91368
Epoch 22209/50000
1/1 [==============================] - 12s 12s/step - loss: 13.9996 - acc: 0.0000e+00 - val_loss: 12.5109 - val_acc: 0.0000e+00

Epoch 22209: val_loss did not improve from 1.91368
Epoch 22210/50000
1/1 [==============================] - 14s 14s/step - loss: 8.8594 - acc: 0.0000e+00 - val_loss: 9.6364 - val_acc: 0.0000e+00

Epoch 22210: val_loss did not improve from 1.91368
Epoch 22211/50000
1/1 [==============================] - 12s 12s/step - loss: 3.0987 - acc: 0.0000e+00 - val_loss: 14.7090 - val_acc: 0.0000e+00

Epoch 22211: va

KeyboardInterrupt: 

In [ ]:
# g=datagen(data_partition='test', batch_size=1)
# x,y = next(g)

In [ ]:
used_words

In [ ]:
# x['the_input'][0].shape, x['source_str'], x['the_labels']


In [ ]:
# # im = batch_frames[1][25]
# import matplotlib.pyplot as plt
# %matplotlib inline
# # generator = datagen(min_row_number=0,max_row_number=200000, batch_size= 6)
# # X,Y = next(generator)
# X = x['the_input'][0]
# # print(X.shape, Y.shape)
# def plot_data(x,ax):
# #     x = x.reshape((size,size))
#     ax.imshow(x, cmap='gray')
# #     if y is not None:
# #         ax.scatter(y[0::2] , y[1::2] , marker='x', s=10)
# fig = plt.figure(figsize=(30,30))
# fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
# for i in range(30):
#     ax = fig.add_subplot(1, 30, i + 1, xticks=[], yticks=[])
#     plot_data(X[i], ax)

In [ ]:
# # import io
# # import base64
# # from IPython.display import HTML

# # video = io.open('evaluation/samples/test.mp4', 'r+b').read()
# # encoded = base64.b64encode(video)
# # HTML(data='''<video alt="test" controls>
# #                 <source src="data:video/mp4;base64,{0}" type="video/mp4" />
# #              </video>'''.format(encoded.decode('ascii')))
# im = cv2.imread('training/unseen_speakers/test_pic.jpg')
# ims = get_frames_mouth(detector, predictor, [im])
# plt.imshow(ims[0][...,::-1])

In [ ]:
# # np.max([len(word) for word in words])
# type(np.array([1,2]))

In [ ]:
# lipnet.predict(np.array([X]))

In [ ]:
# m = []
# m.append([[1,1],[1,1],[1,1]])
# m.append([[1,1],[1,1],[1,1]])
# m.append([[1,1],[1,1],[1,1]])
# m.append([[1,1],[1,1],[1,1]])
# m.append([[1,1],[1,1],[1,1]])
# m.append(1)
# # m.append([1,1,1])
# # m.append([1,1,1])
# # m.append([1,1,1])
# # m.append([1,1,1])
# # m.append([[1],[1],[1]])
# np.array(m).shape
t = {1,2,3,4,5}
t.add(5)
t.add(6)

In [ ]:
t